# 🚗 Büyük Veri ve Analitiği - Dönem Projesi
## Uçtan Uca Data Pipeline: US Accidents Dataset

---

**Proje Bileşenleri:**
1. Veri Alma (Kaggle API)
2. Veri Temizleme (PySpark)
3. kNN Sınıflandırma + Performans Metrikleri
4. K-Means Kümeleme + Görselleştirme
5. Çıktıların JSON olarak dışa aktarılması
6. MongoDB'ye veri yükleme (Lokal PC'de)

**Teknolojiler:** Google Colab, PySpark, MongoDB (Lokal), Python

---

## 📦 Bölüm 1: Kurulum ve Kütüphaneler

In [ ]:
# Gerekli kütüphanelerin kurulumu
!pip install pyspark
!pip install kaggle
!pip install pandas matplotlib seaborn scikit-learn

In [ ]:
# Kütüphaneleri import etme
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, isnan, isnull, mean, stddev, hour, dayofweek, month, year
from pyspark.sql.types import IntegerType, DoubleType, StringType, NumericType
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

# Scikit-learn
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier as SklearnKNN
from sklearn.preprocessing import StandardScaler as SklearnScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, silhouette_score
from sklearn.preprocessing import label_binarize

# Google Colab dosya indirme
from google.colab import files

print("✅ Tüm kütüphaneler başarıyla import edildi!")

## 🔽 Bölüm 2: Veri Alma (Data Ingestion)

Kaggle API kullanarak US Accidents veri setini indireceğiz.

In [ ]:
# Kaggle API Kurulumu
# 1. Kaggle hesabınızdan API token indirin (kaggle.json)
# 2. Aşağıdaki hücreyi çalıştırın ve kaggle.json dosyasını yükleyin

# Kaggle klasörünü oluştur
!mkdir -p ~/.kaggle

print("📁 Lütfen kaggle.json dosyanızı yükleyin:")
print("(Kaggle > Settings > API > Create New Token)")
uploaded = files.upload()

In [ ]:
# Kaggle API yapılandırması
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle API yapılandırıldı!")

In [ ]:
# US Accidents veri setini indir
print("📥 US Accidents veri seti indiriliyor...")
print("(Bu işlem birkaç dakika sürebilir - ~1.2 GB)")

!kaggle datasets download -d sobhanmoosavi/us-accidents

# Zip dosyasını çıkart
!unzip -o us-accidents.zip

print("\n✅ Veri seti başarıyla indirildi!")

In [ ]:
# Dosya boyutunu kontrol et
import os

# CSV dosyasını bul
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
print("📁 Bulunan CSV dosyaları:")
for f in csv_files:
    size_mb = os.path.getsize(f) / (1024 * 1024)
    print(f"   - {f}: {size_mb:.2f} MB")

## 🔥 Bölüm 3: PySpark Oturumu Başlatma

In [ ]:
# Spark oturumu oluştur
spark = SparkSession.builder \
    .appName("US_Accidents_BigData_Pipeline") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

# Spark versiyonunu göster
print(f"✅ Spark Versiyonu: {spark.version}")
print(f"✅ Spark UI: {spark.sparkContext.uiWebUrl}")

In [ ]:
# Veriyi PySpark DataFrame olarak yükle
print("📊 Veri yükleniyor...")

# CSV dosyasını bul ve yükle
csv_file = [f for f in os.listdir('.') if f.endswith('.csv') and 'accident' in f.lower()][0]
print(f"📁 Yüklenen dosya: {csv_file}")

df_spark = spark.read.csv(csv_file, header=True, inferSchema=True)

print(f"\n✅ Veri başarıyla yüklendi!")
print(f"📊 Toplam kayıt sayısı: {df_spark.count():,}")
print(f"📊 Toplam sütun sayısı: {len(df_spark.columns)}")

In [ ]:
# Veri şemasını görüntüle
print("📋 Veri Şeması:")
print("="*60)
df_spark.printSchema()

In [ ]:
# İlk 5 kaydı görüntüle
print("📋 İlk 5 Kayıt:")
df_spark.show(5, truncate=False)

In [ ]:
# Temel istatistikler
print("📊 Temel İstatistikler:")
df_spark.describe().show()

## 🧹 Bölüm 4: Veri Temizleme ve Ön İşleme (PySpark)

In [ ]:
# Eksik değer analizi (OPTİMİZE EDİLMİŞ - tek seferde hesaplama)
print("📊 Eksik Değer Analizi:")
print("="*60)

# Toplam satır sayısı
total_rows = df_spark.count()
print(f"📊 Toplam satır: {total_rows:,}")

# TÜM sütunların null değerlerini TEK SEFERDE hesapla (çok daha hızlı!)
from pyspark.sql.functions import sum as spark_sum, when, col, isnan

# Her sütun için null sayısını hesapla
null_counts_expr = []
for column in df_spark.columns:
    # isNull kontrolü (tüm tipler için)
    null_counts_expr.append(
        spark_sum(when(col(column).isNull(), 1).otherwise(0)).alias(column)
    )

# Tek seferde tüm null'ları say
null_counts_row = df_spark.select(null_counts_expr).collect()[0]

# Sonuçları DataFrame'e çevir
null_counts = []
for column in df_spark.columns:
    null_count = null_counts_row[column]
    if null_count > 0:
        null_percent = (null_count / total_rows) * 100
        null_counts.append((column, null_count, null_percent))

# Eksik değerleri göster
null_df = pd.DataFrame(null_counts, columns=['Sütun', 'Eksik Sayısı', 'Eksik %'])
null_df = null_df.sort_values('Eksik %', ascending=False)
print(f"\n📋 Eksik değer içeren {len(null_df)} sütun bulundu:")
print(null_df.head(20).to_string(index=False))

In [ ]:
# Analiz için önemli sütunları seç
# Sınıflandırma hedefi: Severity (1-4)
# Kümeleme özellikleri: Coğrafi ve çevresel faktörler

selected_columns = [
    'Severity',           # Hedef değişken (1-4)
    'Start_Lat',          # Enlem
    'Start_Lng',          # Boylam
    'Distance(mi)',       # Etkilenen mesafe
    'Temperature(F)',     # Sıcaklık
    'Humidity(%)',        # Nem
    'Pressure(in)',       # Basınç
    'Visibility(mi)',     # Görüş mesafesi
    'Wind_Speed(mph)',    # Rüzgar hızı
    'Precipitation(in)',  # Yağış
    'Weather_Condition',  # Hava durumu
    'Sunrise_Sunset',     # Gündüz/Gece
    'Civil_Twilight',     # Alacakaranlık
    'State'               # Eyalet
]

# Seçili sütunlarla yeni DataFrame oluştur
df_selected = df_spark.select(selected_columns)
print(f"✅ {len(selected_columns)} sütun seçildi.")
df_selected.show(5)

In [ ]:
# Veri temizleme işlemleri
print("🧹 Veri Temizleme Başlıyor...")
print("="*60)

# 1. Eksik değerleri temizle (sadece sayısal sütunlar için ortalama ile doldur)
numeric_columns = ['Start_Lat', 'Start_Lng', 'Distance(mi)', 'Temperature(F)', 
                   'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 
                   'Wind_Speed(mph)', 'Precipitation(in)']

# Ortalama değerleri hesapla ve doldur
df_cleaned = df_selected

for col_name in numeric_columns:
    mean_val = df_cleaned.select(mean(col(col_name))).collect()[0][0]
    if mean_val is not None:
        df_cleaned = df_cleaned.fillna({col_name: mean_val})

# 2. Kategorik değişkenlerdeki eksik değerleri 'Unknown' ile doldur
categorical_columns = ['Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 'State']
for col_name in categorical_columns:
    df_cleaned = df_cleaned.fillna({col_name: 'Unknown'})

# 3. Severity değeri null olan satırları kaldır (hedef değişken)
df_cleaned = df_cleaned.filter(col('Severity').isNotNull())

print(f"✅ Temizleme sonrası kayıt sayısı: {df_cleaned.count():,}")

In [ ]:
# Kategorik değişkenleri sayısallaştır
print("🔄 Kategorik değişkenler sayısallaştırılıyor...")

# StringIndexer ile kategorik değişkenleri encode et
indexers = []
for col_name in categorical_columns:
    indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_Index", handleInvalid="keep")
    indexers.append(indexer)

# Pipeline oluştur ve uygula
indexer_pipeline = Pipeline(stages=indexers)
df_indexed = indexer_pipeline.fit(df_cleaned).transform(df_cleaned)

print("✅ Kategorik değişkenler encode edildi!")
df_indexed.select([f"{c}_Index" for c in categorical_columns]).show(5)

In [ ]:
# Google Colab bellek yönetimi için veriyi örnekle
# Tam veri seti çok büyük olduğundan, analiz için örnek alıyoruz

SAMPLE_SIZE = 500000  # 500K kayıt (belleğe göre ayarlayabilirsiniz)

print(f"📊 Analiz için {SAMPLE_SIZE:,} kayıt örnekleniyor...")
df_sample = df_indexed.sample(fraction=SAMPLE_SIZE/df_indexed.count(), seed=42)
df_sample = df_sample.limit(SAMPLE_SIZE)

print(f"✅ Örnek veri boyutu: {df_sample.count():,} kayıt")

In [ ]:
# Hedef değişken dağılımı (Severity)
print("📊 Severity Dağılımı:")
severity_dist = df_sample.groupBy('Severity').count().orderBy('Severity')
severity_dist.show()

# Görselleştirme için Pandas'a çevir
severity_pd = severity_dist.toPandas()

plt.figure(figsize=(10, 6))
plt.bar(severity_pd['Severity'].astype(str), severity_pd['count'], color=['green', 'yellow', 'orange', 'red'])
plt.xlabel('Kaza Şiddeti (Severity)')
plt.ylabel('Kayıt Sayısı')
plt.title('Kaza Şiddeti Dağılımı')
for i, v in enumerate(severity_pd['count']):
    plt.text(i, v + 1000, f'{v:,}', ha='center', fontsize=10)
plt.tight_layout()
plt.savefig('severity_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 🎯 Bölüm 5: kNN Sınıflandırma + Performans Metrikleri

Kaza şiddetini (Severity 1-4) tahmin etmek için kNN algoritması kullanacağız.

In [ ]:
# Özellik vektörü oluştur
feature_columns = [
    'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Temperature(F)',
    'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
    'Precipitation(in)', 'Weather_Condition_Index', 'Sunrise_Sunset_Index',
    'Civil_Twilight_Index', 'State_Index'
]

# Eksik değerleri tekrar kontrol et ve temizle
for col_name in feature_columns:
    df_sample = df_sample.filter(col(col_name).isNotNull())

print(f"✅ Temiz veri sayısı: {df_sample.count():,}")

In [ ]:
# PySpark'ta kNN doğrudan desteklenmediği için sklearn kullanacağız
# Önce veriyi Pandas'a çevirelim

print("📊 Veri sklearn için hazırlanıyor...")

# Daha küçük bir örnek al (bellek için)
KNN_SAMPLE_SIZE = 100000
df_knn_sample = df_sample.limit(KNN_SAMPLE_SIZE)

# Pandas'a çevir
df_pd = df_knn_sample.select(feature_columns + ['Severity']).toPandas()

print(f"✅ Pandas DataFrame boyutu: {df_pd.shape}")
df_pd.head()

In [ ]:
# Özellik ve hedef değişkenleri ayır
X = df_pd[feature_columns].values
y = df_pd['Severity'].values

# Eksik değerleri kontrol et ve temizle
mask = ~np.isnan(X).any(axis=1)
X = X[mask]
y = y[mask]

print(f"✅ Özellik matrisi boyutu: {X.shape}")
print(f"✅ Hedef vektör boyutu: {y.shape}")
print(f"\n📊 Sınıf dağılımı:")
unique, counts = np.unique(y, return_counts=True)
for u, c in zip(unique, counts):
    print(f"   Severity {int(u)}: {c:,} ({c/len(y)*100:.2f}%)")

In [ ]:
# Train/Test bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"✅ Eğitim seti: {X_train.shape[0]:,} kayıt")
print(f"✅ Test seti: {X_test.shape[0]:,} kayıt")

In [ ]:
# Özellik ölçekleme (StandardScaler)
scaler = SklearnScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Özellikler ölçeklendi (StandardScaler)")

In [ ]:
# kNN Modeli Eğitimi
print("🎯 kNN Modeli Eğitiliyor...")
print("="*60)

# Farklı k değerleri için test
k_values = [3, 5, 7, 9, 11]
results = []

for k in k_values:
    print(f"\n🔄 k={k} için model eğitiliyor...")
    
    knn = SklearnKNN(n_neighbors=k, weights='distance', n_jobs=-1)
    knn.fit(X_train_scaled, y_train)
    
    # Tahmin
    y_pred = knn.predict(X_test_scaled)
    
    # Accuracy hesapla
    accuracy = np.mean(y_pred == y_test)
    results.append({'k': k, 'accuracy': accuracy})
    
    print(f"   Accuracy: {accuracy:.4f}")

# En iyi k değerini bul
best_result = max(results, key=lambda x: x['accuracy'])
print(f"\n🏆 En İyi k değeri: {best_result['k']} (Accuracy: {best_result['accuracy']:.4f})")

In [ ]:
# En iyi k ile final model
BEST_K = best_result['k']

print(f"🎯 Final kNN Modeli (k={BEST_K})")
print("="*60)

knn_final = SklearnKNN(n_neighbors=BEST_K, weights='distance', n_jobs=-1)
knn_final.fit(X_train_scaled, y_train)

# Tahminler
y_pred_final = knn_final.predict(X_test_scaled)
y_pred_proba = knn_final.predict_proba(X_test_scaled)

In [ ]:
# PERFORMANS METRİKLERİ
print("📊 PERFORMANS METRİKLERİ")
print("="*60)

# Classification Report
print("\n📋 Classification Report:")
class_report = classification_report(y_test, y_pred_final, target_names=['Severity 1', 'Severity 2', 'Severity 3', 'Severity 4'])
print(class_report)

In [ ]:
# Detaylı Metrikler
accuracy = accuracy_score(y_test, y_pred_final)
precision_macro = precision_score(y_test, y_pred_final, average='macro')
precision_weighted = precision_score(y_test, y_pred_final, average='weighted')
recall_macro = recall_score(y_test, y_pred_final, average='macro')
recall_weighted = recall_score(y_test, y_pred_final, average='weighted')
f1_macro = f1_score(y_test, y_pred_final, average='macro')
f1_weighted = f1_score(y_test, y_pred_final, average='weighted')

print("📊 ÖZET METRİKLER:")
print("="*60)
print(f"✅ Accuracy:           {accuracy:.4f}")
print(f"✅ Precision (Macro):  {precision_macro:.4f}")
print(f"✅ Precision (Weighted): {precision_weighted:.4f}")
print(f"✅ Recall (Macro):     {recall_macro:.4f}")
print(f"✅ Recall (Weighted):  {recall_weighted:.4f}")
print(f"✅ F1-Score (Macro):   {f1_macro:.4f}")
print(f"✅ F1-Score (Weighted): {f1_weighted:.4f}")

# Metrikleri kaydet
knn_metrics = {
    'model': 'kNN',
    'k_value': int(BEST_K),
    'accuracy': float(accuracy),
    'precision_macro': float(precision_macro),
    'precision_weighted': float(precision_weighted),
    'recall_macro': float(recall_macro),
    'recall_weighted': float(recall_weighted),
    'f1_macro': float(f1_macro),
    'f1_weighted': float(f1_weighted),
    'test_size': int(len(y_test)),
    'train_size': int(len(y_train))
}

In [ ]:
# Confusion Matrix Görselleştirme
cm = confusion_matrix(y_test, y_pred_final)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Severity 1', 'Severity 2', 'Severity 3', 'Severity 4'],
            yticklabels=['Severity 1', 'Severity 2', 'Severity 3', 'Severity 4'])
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.title(f'kNN Confusion Matrix (k={BEST_K})')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Confusion matrix'i metriklere ekle
knn_metrics['confusion_matrix'] = cm.tolist()

In [ ]:
# ROC-AUC Hesaplama (One-vs-Rest)
# Çok sınıflı ROC için etiketleri binarize et
classes = np.unique(y_test)
y_test_bin = label_binarize(y_test, classes=classes)
n_classes = y_test_bin.shape[1]

# Her sınıf için ROC eğrisi
plt.figure(figsize=(12, 8))

colors = ['blue', 'green', 'orange', 'red']
auc_scores = {}

for i, (class_val, color) in enumerate(zip(classes, colors)):
    if i < y_pred_proba.shape[1]:
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        auc_scores[f'Severity_{int(class_val)}'] = float(roc_auc)
        plt.plot(fpr, tpr, color=color, lw=2,
                 label=f'Severity {int(class_val)} (AUC = {roc_auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - kNN Multi-class Classification')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()

# Ortalama AUC
mean_auc = np.mean(list(auc_scores.values()))
print(f"\n📊 Ortalama AUC-ROC: {mean_auc:.4f}")
knn_metrics['auc_roc_mean'] = float(mean_auc)
knn_metrics['auc_roc_per_class'] = auc_scores

In [ ]:
# k değerlerine göre accuracy grafiği
results_df = pd.DataFrame(results)

plt.figure(figsize=(10, 6))
plt.plot(results_df['k'], results_df['accuracy'], 'bo-', linewidth=2, markersize=10)
plt.xlabel('k Değeri')
plt.ylabel('Accuracy')
plt.title('kNN: k Değerine Göre Accuracy')
plt.xticks(k_values)
plt.grid(True, alpha=0.3)

# En iyi k'yı işaretle
plt.axvline(x=BEST_K, color='red', linestyle='--', label=f'En İyi k={BEST_K}')
plt.legend()
plt.tight_layout()
plt.savefig('knn_k_accuracy.png', dpi=150, bbox_inches='tight')
plt.show()

## 🎨 Bölüm 6: K-Means Kümeleme + Görselleştirme

In [ ]:
# K-Means için özellik seçimi
# Coğrafi ve çevresel faktörlere göre kazaları kümeleyeceğiz

kmeans_features = [
    'Start_Lat', 'Start_Lng', 'Temperature(F)', 'Humidity(%)',
    'Visibility(mi)', 'Wind_Speed(mph)'
]

# PySpark VectorAssembler ile özellik vektörü oluştur
assembler = VectorAssembler(inputCols=kmeans_features, outputCol="features", handleInvalid="skip")
df_kmeans = assembler.transform(df_sample)

# Ölçekleme
scaler_spark = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler_spark.fit(df_kmeans)
df_kmeans_scaled = scaler_model.transform(df_kmeans)

print("✅ K-Means için özellikler hazırlandı!")

In [ ]:
# Elbow Method - Optimal k değerini bul
print("📊 Elbow Method ile Optimal k Belirleme...")
print("="*60)

# Daha küçük bir örnekle çalış (hız için)
df_kmeans_sample = df_kmeans_scaled.sample(fraction=0.1, seed=42)

k_range = range(2, 11)
costs = []

for k in k_range:
    print(f"🔄 k={k} test ediliyor...")
    
    kmeans = KMeans(k=k, seed=42, featuresCol="scaled_features", predictionCol="cluster")
    model = kmeans.fit(df_kmeans_sample)
    
    # WSSSE (Within Set Sum of Squared Errors)
    cost = model.summary.trainingCost
    costs.append(cost)
    
    print(f"   WSSSE: {cost:.2f}")

print("\n✅ Elbow analizi tamamlandı!")

In [ ]:
# Elbow grafiği
plt.figure(figsize=(10, 6))
plt.plot(list(k_range), costs, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('WSSSE (Within Set Sum of Squared Errors)')
plt.title('K-Means: Elbow Method')
plt.xticks(list(k_range))
plt.grid(True, alpha=0.3)

# Optimal k'yı işaretle (genellikle dirsek noktası)
optimal_k = 4  # Görsel incelemeye göre ayarlayın
plt.axvline(x=optimal_k, color='red', linestyle='--', label=f'Optimal k={optimal_k}')
plt.legend()
plt.tight_layout()
plt.savefig('elbow_method.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Final K-Means Modeli
OPTIMAL_K = 4

print(f"🎯 Final K-Means Modeli (k={OPTIMAL_K})")
print("="*60)

kmeans_final = KMeans(k=OPTIMAL_K, seed=42, featuresCol="scaled_features", predictionCol="cluster")
kmeans_model = kmeans_final.fit(df_kmeans_scaled)

# Küme atamaları
df_clustered = kmeans_model.transform(df_kmeans_scaled)

# Küme merkezleri
centers = kmeans_model.clusterCenters()
print("\n📍 Küme Merkezleri:")
for i, center in enumerate(centers):
    print(f"   Küme {i}: {center[:3]}... (ilk 3 özellik)")

In [ ]:
# Küme dağılımı
print("\n📊 Küme Dağılımı:")
cluster_dist = df_clustered.groupBy('cluster').count().orderBy('cluster')
cluster_dist.show()

cluster_pd = cluster_dist.toPandas()

plt.figure(figsize=(10, 6))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
plt.bar(cluster_pd['cluster'].astype(str), cluster_pd['count'], color=colors[:len(cluster_pd)])
plt.xlabel('Küme')
plt.ylabel('Kayıt Sayısı')
plt.title('K-Means Küme Dağılımı')
for i, v in enumerate(cluster_pd['count']):
    plt.text(i, v + 500, f'{v:,}', ha='center', fontsize=10)
plt.tight_layout()
plt.savefig('cluster_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Kümelerin coğrafi görselleştirmesi
print("🗺️ Coğrafi Küme Görselleştirmesi...")

# Görselleştirme için örnek al
df_viz = df_clustered.select('Start_Lat', 'Start_Lng', 'cluster', 'Severity').limit(50000).toPandas()

plt.figure(figsize=(16, 10))
scatter = plt.scatter(df_viz['Start_Lng'], df_viz['Start_Lat'], 
                      c=df_viz['cluster'], cmap='viridis', alpha=0.5, s=1)
plt.colorbar(scatter, label='Küme')
plt.xlabel('Boylam (Longitude)')
plt.ylabel('Enlem (Latitude)')
plt.title('ABD Trafik Kazaları - K-Means Kümeleme (Coğrafi)')
plt.tight_layout()
plt.savefig('geographic_clusters.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Küme karakteristikleri analizi
print("📊 Küme Karakteristikleri:")
print("="*60)

cluster_stats = df_clustered.groupBy('cluster').agg(
    mean('Temperature(F)').alias('Avg_Temp'),
    mean('Humidity(%)').alias('Avg_Humidity'),
    mean('Visibility(mi)').alias('Avg_Visibility'),
    mean('Wind_Speed(mph)').alias('Avg_Wind'),
    mean('Severity').alias('Avg_Severity'),
    count('*').alias('Count')
).orderBy('cluster')

cluster_stats.show()

In [ ]:
# Küme karakteristikleri görselleştirmesi
cluster_stats_pd = cluster_stats.toPandas()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Ortalama Sıcaklık
axes[0, 0].bar(cluster_stats_pd['cluster'].astype(str), cluster_stats_pd['Avg_Temp'], color='coral')
axes[0, 0].set_xlabel('Küme')
axes[0, 0].set_ylabel('Ortalama Sıcaklık (°F)')
axes[0, 0].set_title('Kümelere Göre Ortalama Sıcaklık')

# Ortalama Nem
axes[0, 1].bar(cluster_stats_pd['cluster'].astype(str), cluster_stats_pd['Avg_Humidity'], color='skyblue')
axes[0, 1].set_xlabel('Küme')
axes[0, 1].set_ylabel('Ortalama Nem (%)')
axes[0, 1].set_title('Kümelere Göre Ortalama Nem')

# Ortalama Görüş Mesafesi
axes[1, 0].bar(cluster_stats_pd['cluster'].astype(str), cluster_stats_pd['Avg_Visibility'], color='lightgreen')
axes[1, 0].set_xlabel('Küme')
axes[1, 0].set_ylabel('Ortalama Görüş (mi)')
axes[1, 0].set_title('Kümelere Göre Ortalama Görüş Mesafesi')

# Ortalama Kaza Şiddeti
axes[1, 1].bar(cluster_stats_pd['cluster'].astype(str), cluster_stats_pd['Avg_Severity'], color='salmon')
axes[1, 1].set_xlabel('Küme')
axes[1, 1].set_ylabel('Ortalama Şiddet')
axes[1, 1].set_title('Kümelere Göre Ortalama Kaza Şiddeti')

plt.tight_layout()
plt.savefig('cluster_characteristics.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Silhouette Score hesaplama (sklearn ile)
# Küçük bir örnekle silhouette hesapla
df_sil = df_clustered.select(kmeans_features + ['cluster']).limit(20000).toPandas()
df_sil = df_sil.dropna()

X_sil = df_sil[kmeans_features].values
y_sil = df_sil['cluster'].values

sil_score = silhouette_score(X_sil, y_sil)
print(f"📊 Silhouette Score: {sil_score:.4f}")

# K-Means metriklerini kaydet
kmeans_metrics = {
    'model': 'K-Means',
    'k_value': int(OPTIMAL_K),
    'wssse': float(kmeans_model.summary.trainingCost),
    'silhouette_score': float(sil_score),
    'cluster_sizes': cluster_pd.to_dict('records'),
    'cluster_centers': [center.tolist() for center in centers],
    'cluster_stats': cluster_stats_pd.to_dict('records')
}

## 💾 Bölüm 7: Çıktıları JSON Olarak Dışa Aktarma

Tüm sonuçları JSON dosyaları olarak kaydedip bilgisayarınıza indireceğiz.
Bu dosyaları daha sonra lokal MongoDB'ye yükleyeceksiniz.

In [ ]:
# Çıktı klasörü oluştur
import os
OUTPUT_DIR = 'mongodb_export'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Çıktı klasörü oluşturuldu: {OUTPUT_DIR}")

In [ ]:
# 1. kNN Sonuçlarını kaydet
knn_metrics['timestamp'] = datetime.now().isoformat()

with open(f'{OUTPUT_DIR}/knn_results.json', 'w', encoding='utf-8') as f:
    json.dump(knn_metrics, f, indent=2, ensure_ascii=False)

print("✅ kNN sonuçları kaydedildi: knn_results.json")

In [ ]:
# 2. K-Means Sonuçlarını kaydet
kmeans_metrics['timestamp'] = datetime.now().isoformat()

with open(f'{OUTPUT_DIR}/kmeans_results.json', 'w', encoding='utf-8') as f:
    json.dump(kmeans_metrics, f, indent=2, ensure_ascii=False)

print("✅ K-Means sonuçları kaydedildi: kmeans_results.json")

In [ ]:
# 3. Ham veri örneğini kaydet (ilk 10,000 kayıt)
print("📊 Ham veri örneği hazırlanıyor...")

raw_sample = df_spark.limit(10000).toPandas()
raw_sample = raw_sample.where(pd.notnull(raw_sample), None)

# Datetime sütunlarını string'e çevir
for col in raw_sample.columns:
    if raw_sample[col].dtype == 'datetime64[ns]':
        raw_sample[col] = raw_sample[col].astype(str)

raw_records = raw_sample.to_dict('records')

with open(f'{OUTPUT_DIR}/us_accidents_raw.json', 'w', encoding='utf-8') as f:
    json.dump(raw_records, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Ham veri kaydedildi: us_accidents_raw.json ({len(raw_records):,} kayıt)")

In [ ]:
# 4. Temizlenmiş veri örneğini kaydet (ilk 10,000 kayıt)
print("📊 Temizlenmiş veri örneği hazırlanıyor...")

cleaned_sample = df_sample.limit(10000).toPandas()
cleaned_sample = cleaned_sample.where(pd.notnull(cleaned_sample), None)

# Tüm sütunları uygun tipe çevir
for col in cleaned_sample.columns:
    if cleaned_sample[col].dtype == 'datetime64[ns]':
        cleaned_sample[col] = cleaned_sample[col].astype(str)

cleaned_records = cleaned_sample.to_dict('records')

with open(f'{OUTPUT_DIR}/us_accidents_cleaned.json', 'w', encoding='utf-8') as f:
    json.dump(cleaned_records, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Temizlenmiş veri kaydedildi: us_accidents_cleaned.json ({len(cleaned_records):,} kayıt)")

In [ ]:
# 5. Kümelenmiş veri örneğini kaydet
print("📊 Kümelenmiş veri örneği hazırlanıyor...")

clustered_sample = df_clustered.select(
    'Start_Lat', 'Start_Lng', 'Temperature(F)', 'Humidity(%)',
    'Visibility(mi)', 'Wind_Speed(mph)', 'Severity', 'cluster'
).limit(10000).toPandas()

clustered_sample = clustered_sample.where(pd.notnull(clustered_sample), None)
clustered_records = clustered_sample.to_dict('records')

with open(f'{OUTPUT_DIR}/us_accidents_clustered.json', 'w', encoding='utf-8') as f:
    json.dump(clustered_records, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Kümelenmiş veri kaydedildi: us_accidents_clustered.json ({len(clustered_records):,} kayıt)")

In [ ]:
# Tüm dosyaları listele
print("\n📁 Oluşturulan Dosyalar:")
print("="*60)

for f in os.listdir(OUTPUT_DIR):
    size_kb = os.path.getsize(f'{OUTPUT_DIR}/{f}') / 1024
    print(f"   📄 {f}: {size_kb:.2f} KB")

In [ ]:
# Dosyaları ZIP olarak paketle
import shutil

# Görsel dosyalarını da ekle
image_files = ['severity_distribution.png', 'confusion_matrix.png', 'roc_curves.png', 
               'knn_k_accuracy.png', 'elbow_method.png', 'cluster_distribution.png',
               'geographic_clusters.png', 'cluster_characteristics.png']

for img in image_files:
    if os.path.exists(img):
        shutil.copy(img, OUTPUT_DIR)

# ZIP dosyası oluştur
shutil.make_archive('bigdata_project_output', 'zip', OUTPUT_DIR)

print("\n✅ Tüm dosyalar ZIP olarak paketlendi: bigdata_project_output.zip")

In [ ]:
# ZIP dosyasını indir
print("📥 ZIP dosyası indiriliyor...")
files.download('bigdata_project_output.zip')

print("\n✅ İndirme başlatıldı!")
print("\n📋 Sonraki Adımlar:")
print("   1. ZIP dosyasını bilgisayarınıza indirin")
print("   2. ZIP dosyasını çıkartın")
print("   3. Lokal MongoDB'yi başlatın")
print("   4. 'mongodb_local_import.py' scriptini çalıştırın")

## 📊 Bölüm 8: Özet ve Sonuçlar

In [ ]:
# PROJE ÖZETİ
print("="*70)
print("                    📊 PROJE ÖZETİ                    ")
print("="*70)

print("\n📁 VERİ SETİ:")
print(f"   • Kaynak: US Accidents Dataset (Kaggle)")
print(f"   • Toplam Kayıt: ~7.7 milyon")
print(f"   • Analiz için Kullanılan: {KNN_SAMPLE_SIZE:,} kayıt")
print(f"   • Özellik Sayısı: {len(feature_columns)}")

print("\n🎯 kNN SINIFLANDIRMA SONUÇLARI:")
print(f"   • En İyi k Değeri: {BEST_K}")
print(f"   • Accuracy: {accuracy:.4f}")
print(f"   • Precision (Weighted): {precision_weighted:.4f}")
print(f"   • Recall (Weighted): {recall_weighted:.4f}")
print(f"   • F1-Score (Weighted): {f1_weighted:.4f}")
print(f"   • AUC-ROC (Ortalama): {mean_auc:.4f}")

print("\n🎨 K-MEANS KÜMELEME SONUÇLARI:")
print(f"   • Optimal Küme Sayısı: {OPTIMAL_K}")
print(f"   • WSSSE: {kmeans_model.summary.trainingCost:.2f}")
print(f"   • Silhouette Score: {sil_score:.4f}")

print("\n💾 DIŞA AKTARILAN DOSYALAR:")
print(f"   • knn_results.json")
print(f"   • kmeans_results.json")
print(f"   • us_accidents_raw.json")
print(f"   • us_accidents_cleaned.json")
print(f"   • us_accidents_clustered.json")
print(f"   • Görselleştirme PNG dosyaları")

print("\n" + "="*70)
print("        ✅ COLAB KISMI TAMAMLANDI - DOSYALARI İNDİRİN        ")
print("="*70)

In [ ]:
# Spark oturumunu kapat
spark.stop()
print("\n✅ Spark oturumu kapatıldı.")